# 📊 Proyecto: Análisis y visualización de la calidad del aire: una exploración de contaminantes atmosféricos y su relación con el PM2.5
**Curso:** Samsung Innovation Campus – Módulo de Python (Ecuador 2025)  
**Seccion:** EC03  

---

## 🧩 Módulo: Adquisición de Datos

En esta sección se carga el dataset, se explora su estructura general y se validan los rangos de los contaminantes.

El dataset contiene mediciones de diferentes gases y partículas contaminantes en el aire, registradas con fecha y hora.


### 📘 Descripción general de las columnas del dataset

| Columna | Descripción | Relevancia |
|----------|--------------|-------------|
| 📅 **Date** | Fecha y hora del registro | Permite analizar variaciones temporales. |
| 🏭 **CO** | Monóxido de carbono (ppm) | Gas tóxico, indicador de combustión incompleta. |
| 🚗 **NO** | Óxido nítrico | Se genera en la quema de combustibles fósiles. |
| 🚙 **NO2** | Dióxido de nitrógeno | Contaminante urbano, precursor del ozono. |
| 🌫️ **O3** | Ozono troposférico | Se forma con NOx + luz solar; irritante respiratorio. |
| 🌋 **SO2** | Dióxido de azufre | Proviene de la quema de carbón y petróleo. |
| ☁️ **PM2.5** | Partículas finas < 2.5 μm | Altamente dañinas, ingresan a los pulmones. |
| 🌧️ **PM10** | Partículas < 10 μm | Menos dañinas, quedan en vías respiratorias. |
| 🍃 **NH3** | Amoníaco | De origen agrícola, contribuye a partículas secundarias. |


In [222]:
# ===== Importar librerías principales ====
import pandas as pd
import numpy as np
import warnings
import importlib
import src.data_acquisition as da
import src.data_processing as dp
import src.data_visualization as dv
import src.data_interpretation as di

# Recargar los módulos para reflejar cambios recientes
importlib.reload(dp) 
importlib.reload(da)
importlib.reload(dv)
importlib.reload(di)

warnings.filterwarnings('ignore')

In [223]:
# === Listar los archivos disponibles en la carpeta de datos ===
print("="*60)
print("📂 Archivos disponibles en la carpeta")
print("="*60)
for file in da.list_data_files("data"):
    print(f"- {file}")

📂 Archivos disponibles en la carpeta
- data\delhiaqi.csv


In [224]:
# Cargar de los datos del archivo CSV en un DataFrame
df_air_quality = da.load_data_csv('data\delhiaqi.csv')

✅ Archivo 'data\delhiaqi.csv' cargado exitosamente.


In [225]:
print("=" * 60)
print("📊 Exploración inicial de los datos")
print("=" * 60)

# Mostrar filas del DataFrame
da.preview_data(df_air_quality, num_rows=5)

📊 Exploración inicial de los datos

🔍 Primeras 5 filas del DataFrame:


,date,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2023-01-01 00:00:00,1655.58,1.66,39.41,5.90,17.88,169.29,194.64,5.83
1,2023-01-01 01:00:00,1869.20,6.82,42.16,1.99,22.17,182.84,211.08,7.66
2,2023-01-01 02:00:00,2510.07,27.72,43.87,0.02,30.04,220.25,260.68,11.40
3,2023-01-01 03:00:00,3150.94,55.43,44.55,0.85,35.76,252.90,304.12,13.55
4,2023-01-01 04:00:00,3471.37,68.84,45.24,5.45,39.10,266.36,322.80,14.19


In [226]:
# Mostrar dimensiones del DataFrame
da.get_dataframe_shape(df_air_quality)


📐 Dimensiones del DataFrame: 561 filas y 9 columnas


In [227]:
# Mostrar información detallada del DataFrame
da.get_dataframe_info(df_air_quality)


🧱 Columnas del Dataframe:
['date', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']

📊 Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    561 non-null    object 
 1   co      561 non-null    float64
 2   no      561 non-null    float64
 3   no2     561 non-null    float64
 4   o3      561 non-null    float64
 5   so2     561 non-null    float64
 6   pm2_5   561 non-null    float64
 7   pm10    561 non-null    float64
 8   nh3     561 non-null    float64
dtypes: float64(8), object(1)
memory usage: 39.6+ KB


In [228]:
# Mostrar los valores faltantes por columna
da.get_missing_values(df_air_quality)


❗ Valores faltantes por columna:
date     0
co       0
no       0
no2      0
o3       0
so2      0
pm2_5    0
pm10     0
nh3      0
dtype: int64


## 🧩 Módulo: Procesamiento y limpieza de datos
En esta sección se prepara el conjunto de datos para el análisis.  
Se aplican los siguientes pasos:

1. Conversión de fechas al formato `datetime`.  
2. Eliminación de duplicados y valores negativos.  
3. Creación de columnas temporales (año, mes, día, hora).  
4. Cálculo automático del **Índice de Calidad del Aire (ICA)** basado en PM2.5.
5. Clasificación automática de cada registro en su categoría correspondiente según su valor de PM2.5.

### 🔹 Índice de Calidad del Aire (ICA)
El ICA mide la calidad del aire en una escala de **0 a >500**, dividiéndose en seis categorías de peligrosidad.  
A mayor índice, peor es la calidad del aire.  

| Categoría | Color | Rango ICA |
|-----------|-------|-----------|
| Buena | 🟢 Verde | 0 – 50 |
| Moderada | 🟡 Amarillo | 51 – 100 |
| Dañina para grupos sensibles | 🟠 Naranja | 101 – 150 |
| Dañina | 🔴 Rojo | 151 – 200 |
| Muy dañina | 🟣 Morado | 201 – 300 |
| Peligrosa | 🟤 Marrón | >300 |


In [229]:
# Limpiar y preparar los datos
df_clean = dp.clean_dataframe(df_air_quality)

# Agregar columna de categorías ICA (Índice de Calidad del Aire)
df_clean = dp.add_ica_category(df_clean)

# Mostrar las primera filas procesadas
da.preview_data(df_clean, num_rows=5)

✅ Datos limpios y listos para el análisis.

🔍 Primeras 5 filas del DataFrame:


,co,no,no2,o3,so2,pm2_5,pm10,nh3,year,month,day,hour,ica_category
date,,,,,,,,,,,,,
2023-01-01 00:00:00,1655.58,1.66,39.41,5.90,17.88,169.29,194.64,5.83,2023,1,1,0,Dañina
2023-01-01 01:00:00,1869.20,6.82,42.16,1.99,22.17,182.84,211.08,7.66,2023,1,1,1,Dañina
2023-01-01 02:00:00,2510.07,27.72,43.87,0.02,30.04,220.25,260.68,11.40,2023,1,1,2,Muy dañina
2023-01-01 03:00:00,3150.94,55.43,44.55,0.85,35.76,252.90,304.12,13.55,2023,1,1,3,Muy dañina
2023-01-01 04:00:00,3471.37,68.84,45.24,5.45,39.10,266.36,322.80,14.19,2023,1,1,4,Muy dañina


In [230]:
# Reporte de calidad de datos
reporte = dp.quality_report(df_clean)

print("📋 Reporte de Calidad de Datos:")
for key, value in reporte.items():
    print(f"{key}: {value}")

📋 Reporte de Calidad de Datos:
rows: 561
cols: 13
duplicates: 0
inferred_freq: h
missing: {'co': 0, 'no': 0, 'no2': 0, 'o3': 0, 'so2': 0, 'pm2_5': 0, 'pm10': 0, 'nh3': 0, 'year': 0, 'month': 0, 'day': 0, 'hour': 0, 'ica_category': 0}


In [231]:
# Estadísticas descriptivas
desc = dp.descriptives(df_clean)
print("📊 Estadísticas descriptivas por contaminante:")
display(desc)

📊 Estadísticas descriptivas por contaminante:


,count,mean,std,min,p05,p50,p95,max
co,561.0,3814.942210,3227.744681,654.22,1188.28,2590.18,11428.83,16876.22
no,561.0,51.181979,83.904476,0.00,0.00,13.30,236.03,425.58
no2,561.0,75.292496,42.473791,13.37,26.39,63.75,159.03,263.21
o3,561.0,30.141943,39.979405,0.00,0.00,11.80,124.45,164.51
so2,561.0,64.655936,61.073080,5.25,17.64,47.21,177.38,511.17
pm2_5,561.0,358.256364,227.359117,60.10,128.92,301.17,844.98,1310.20
pm10,561.0,420.988414,271.287026,69.08,158.34,340.90,1035.78,1499.27
nh3,561.0,26.425062,36.563094,0.63,4.31,14.82,102.34,267.51


In [232]:
# Promedios y resúmenes por mes

# Crear instancia de análisis
estacion = dp.EstacionCalidadAire("Delhi", df_clean)

# Promedio mensual de PM2.5
prom_mes_pm25 = estacion.promedio_por_mes("pm2_5")
print("📈 Promedio mensual de PM2.5:")
prom_mes_pm25.head()

📈 Promedio mensual de PM2.5:


,month,pm2_5
0,1,358.256364


In [233]:
# Obtener el máximo global de PM2.5
maximo_mensual_pm25 = estacion.maximo_global("pm2_5")
print(f"🔝 Máximo global de PM2.5: {maximo_mensual_pm25}")

🔝 Máximo global de PM2.5: 1310.2


In [234]:
# Días más contaminados
dias_peores = estacion.top_n_dias_mas_contaminados("pm2_5", n=5)
print("🚨 Días más contaminados por PM2.5:")
display(dias_peores)

🚨 Días más contaminados por PM2.5:


,date,pm2_5
449,2023-01-19 17:00:00,1310.20
308,2023-01-13 20:00:00,1278.35
307,2023-01-13 19:00:00,1232.62
448,2023-01-19 16:00:00,1228.04
306,2023-01-13 18:00:00,1225.39


In [235]:
# Contar registros por categoría ICA (Índice de Calidad del Aire)
print("📊 Distribución de categorías ICA:")
display(df_clean["ica_category"].value_counts())

📊 Distribución de categorías ICA:


ica_category
Peligrosa                       282
Muy dañina                      141
Dañina                           81
Dañina para grupos sensibles     48
Moderada                          9
Name: count, dtype: int64

# Modulo Visualizacion de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Optional, Tuple
import warnings

In [ ]:
# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Colores ICA (importados de tu módulo)
ICA_COLORS = {
    "Buena": "#00E400",
    "Moderada": "#FFFF00",
    "Dañina para grupos sensibles": "#FF7E00",
    "Dañina": "#FF0000",
    "Muy dañina": "#8F3F97",
    "Peligrosa": "#7E0023"
}

CONTAMINANTES = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']

In [ ]:
def plot_time_series(df: pd.DataFrame, columnas: List[str] = ['pm2_5', 'pm10'], figsize: Tuple[int, int] = (14, 6), titulo: str = "Evolución Temporal de Contaminantes") -> None:
    """
    Gráfico de líneas para visualizar la evolución temporal de contaminantes.
    
    Parámetros:
        df: DataFrame con índice datetime
        columnas: Lista de columnas a graficar
        figsize: Tamaño de la figura
        titulo: Título del gráfico
    """
    fig, ax = plt.subplots(figsize=figsize)
    
    for col in columnas:
        if col in df.columns:
            ax.plot(df.index, df[col], label=col.upper(), linewidth=1.5, alpha=0.8)
    
    ax.set_xlabel("Fecha", fontsize=12)
    ax.set_ylabel("Concentración (µg/m³)", fontsize=12)
    ax.set_title(titulo, fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_ica_distribution(df: pd.DataFrame, 
                        figsize: Tuple[int, int] = (12, 5)) -> None:
    """
    Visualiza la distribución de categorías ICA mediante gráfico de barras y pie chart.
    
    Parámetros:
        df: DataFrame con columna 'ica_category'
        figsize: Tamaño de la figura
    """
    if 'ica_category' not in df.columns:
        print("❌ La columna 'ica_category' no existe. Ejecuta add_ica_category() primero.")
        return
    
    # Orden de categorías
    orden_ica = ["Buena", "Moderada", "Dañina para grupos sensibles", "Dañina", "Muy dañina", "Peligrosa"]
    
    # Contar frecuencias
    counts = df['ica_category'].value_counts()
    counts = counts.reindex(orden_ica, fill_value=0)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Gráfico de barras
    colors = [ICA_COLORS.get(cat, '#CCCCCC') for cat in counts.index]
    ax1.bar(range(len(counts)), counts.values, color=colors, edgecolor='black', alpha=0.8)
    ax1.set_xticks(range(len(counts)))
    ax1.set_xticklabels(counts.index, rotation=45, ha='right', fontsize=9)
    ax1.set_ylabel("Frecuencia", fontsize=11)
    ax1.set_title("Distribución de Categorías ICA", fontsize=13, fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    # Gráfico circular
    ax2.pie(counts.values, labels=counts.index, colors=colors, autopct='%1.1f%%',
            startangle=90, textprops={'fontsize': 9})
    ax2.set_title("Proporción de Días por Categoría ICA", fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    plt.show()


def plot_heatmap_hourly(df: pd.DataFrame, columna: str = 'pm2_5', figsize: Tuple[int, int] = (12, 6)) -> None:
    """
    Heatmap mostrando patrones horarios y por día de la semana.
    
    Parámetros:
        df: DataFrame con índice datetime
        columna: Columna a visualizar
        figsize: Tamaño de la figura
    """
    if columna not in df.columns:
        print(f"❌ La columna '{columna}' no existe en el DataFrame.")
        return
    
    # Crear copia con día de semana y hora
    df_temp = df.copy()
    df_temp['dow'] = df_temp.index.dayofweek  # 0=Lunes, 6=Domingo
    df_temp['hour'] = df_temp.index.hour
    
    # Pivotear datos
    pivot = df_temp.pivot_table(values=columna, index='dow', columns='hour', aggfunc='mean')
    
    # Nombres de días
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    pivot.index = [dias[i] for i in pivot.index]
    
    # Crear heatmap
    plt.figure(figsize=figsize)
    sns.heatmap(pivot, cmap='YlOrRd', annot=False, fmt='.1f', cbar_kws={'label': f'{columna.upper()} (µg/m³)'})
    plt.title(f'Patrón Horario de {columna.upper()} por Día de la Semana', fontsize=14, fontweight='bold')
    plt.xlabel('Hora del Día', fontsize=12)
    plt.ylabel('Día de la Semana', fontsize=12)
    plt.tight_layout()
    plt.show()


def plot_monthly_boxplot(df: pd.DataFrame, 
                        columna: str = 'pm2_5',
                        figsize: Tuple[int, int] = (14, 6)) -> None:
    """
    Boxplot mostrando la distribución mensual de un contaminante.
    
    Parámetros:
        df: DataFrame con columna 'month'
        columna: Columna a visualizar
        figsize: Tamaño de la figura
    """
    if columna not in df.columns or 'month' not in df.columns:
        print(f"❌ Falta columna '{columna}' o 'month' en el DataFrame.")
        return
    
    plt.figure(figsize=figsize)
    
    # Preparar datos
    df_plot = df[[columna, 'month']].dropna()
    
    # Crear boxplot
    bp = plt.boxplot([df_plot[df_plot['month'] == m][columna].values 
                    for m in range(1, 13)],
                    labels=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                            'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'],
                    patch_artist=True, showfliers=True)
    
    # Colorear cajas
    colors = plt.cm.viridis(np.linspace(0, 1, 12))
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    plt.xlabel("Mes", fontsize=12)
    plt.ylabel(f"{columna.upper()} (µg/m³)", fontsize=12)
    plt.title(f"Distribución Mensual de {columna.upper()}", fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_correlation_matrix(df: pd.DataFrame, 
                        columnas: Optional[List[str]] = None,
                        figsize: Tuple[int, int] = (10, 8)) -> None:
    """
    Matriz de correlación entre contaminantes.
    
    Parámetros:
        df: DataFrame con datos de contaminantes
        columnas: Lista de columnas a correlacionar (por defecto CONTAMINANTES)
        figsize: Tamaño de la figura
    """
    if columnas is None:
        columnas = [c for c in CONTAMINANTES if c in df.columns]
    else:
        columnas = [c for c in columnas if c in df.columns]
    
    if len(columnas) < 2:
        print("❌ Se necesitan al menos 2 columnas para calcular correlaciones.")
        return
    
    # Calcular correlación
    corr = df[columnas].corr()
    
    # Crear heatmap
    plt.figure(figsize=figsize)
    mask = np.triu(np.ones_like(corr, dtype=bool))  # Máscara triangular
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title("Matriz de Correlación entre Contaminantes", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


def plot_top_n_contaminated_days(df: pd.DataFrame, 
                                columna: str = 'pm2_5',
                                n: int = 10,
                                figsize: Tuple[int, int] = (12, 6)) -> None:
    """
    Gráfico de barras horizontales de los N días más contaminados.
    
    Parámetros:
        df: DataFrame con índice datetime
        columna: Columna a analizar
        n: Número de días a mostrar
        figsize: Tamaño de la figura
    """
    if columna not in df.columns:
        print(f"❌ La columna '{columna}' no existe.")
        return
    
    # Agrupar por día y ordenar
    daily = df.groupby(df.index.date)[columna].mean().sort_values(ascending=False).head(n)
    
    # Crear gráfico
    plt.figure(figsize=figsize)
    colors = plt.cm.Reds(np.linspace(0.4, 0.9, n))
    plt.barh(range(n), daily.values, color=colors, edgecolor='black', alpha=0.8)
    plt.yticks(range(n), [str(d) for d in daily.index])
    plt.xlabel(f"{columna.upper()} (µg/m³)", fontsize=12)
    plt.ylabel("Fecha", fontsize=12)
    plt.title(f"Top {n} Días con Mayor Concentración de {columna.upper()}", 
            fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_distribution_histogram(df: pd.DataFrame, 
                                columna: str = 'pm2_5',
                                bins: int = 50,
                                figsize: Tuple[int, int] = (10, 6)) -> None:
    """
    Histograma con curva KDE mostrando la distribución de un contaminante.
    
    Parámetros:
        df: DataFrame
        columna: Columna a visualizar
        bins: Número de bins del histograma
        figsize: Tamaño de la figura
    """
    if columna not in df.columns:
        print(f"❌ La columna '{columna}' no existe.")
        return
    
    data = df[columna].dropna()
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Histograma
    ax.hist(data, bins=bins, color='skyblue', edgecolor='black', 
            alpha=0.7, density=True, label='Histograma')
    
    # Curva KDE
    data.plot(kind='kde', ax=ax, color='red', linewidth=2, label='KDE')
    
    ax.set_xlabel(f"{columna.upper()} (µg/m³)", fontsize=12)
    ax.set_ylabel("Densidad", fontsize=12)
    ax.set_title(f"Distribución de {columna.upper()}", fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_scatter_comparison(df: pd.DataFrame, 
                        col_x: str = 'pm2_5', 
                        col_y: str = 'pm10',
                        figsize: Tuple[int, int] = (10, 6)) -> None:
    """
    Scatter plot comparando dos contaminantes.
    
    Parámetros:
        df: DataFrame
        col_x: Columna para eje X
        col_y: Columna para eje Y
        figsize: Tamaño de la figura
    """
    if col_x not in df.columns or col_y not in df.columns:
        print(f"❌ Una o ambas columnas no existen en el DataFrame.")
        return
    
    plt.figure(figsize=figsize)
    plt.scatter(df[col_x], df[col_y], alpha=0.5, s=10, c='purple', edgecolors='none')
    plt.xlabel(f"{col_x.upper()} (µg/m³)", fontsize=12)
    plt.ylabel(f"{col_y.upper()} (µg/m³)", fontsize=12)
    plt.title(f"Relación entre {col_x.upper()} y {col_y.upper()}", 
            fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_monthly_averages(df: pd.DataFrame, 
                        columna: str = 'pm2_5',
                        figsize: Tuple[int, int] = (12, 6)) -> None:
    """
    Gráfico de barras de promedios mensuales.
    
    Parámetros:
        df: DataFrame con columna 'month'
        columna: Columna a promediar
        figsize: Tamaño de la figura
    """
    if columna not in df.columns or 'month' not in df.columns:
        print(f"❌ Falta columna '{columna}' o 'month'.")
        return
    
    monthly = df.groupby('month')[columna].mean()
    
    plt.figure(figsize=figsize)
    colors = plt.cm.plasma(np.linspace(0, 1, 12))
    plt.bar(monthly.index, monthly.values, color=colors, edgecolor='black', alpha=0.8)
    plt.xticks(range(1, 13), ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                            'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
    plt.xlabel("Mes", fontsize=12)
    plt.ylabel(f"Promedio de {columna.upper()} (µg/m³)", fontsize=12)
    plt.title(f"Promedio Mensual de {columna.upper()}", fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def create_dashboard_summary(df: pd.DataFrame, 
                            columna_principal: str = 'pm2_5',
                            figsize: Tuple[int, int] = (16, 12)) -> None:
    """
    Dashboard completo con múltiples visualizaciones en una sola figura.
    
    Parámetros:
        df: DataFrame completo
        columna_principal: Contaminante principal a analizar
        figsize: Tamaño de la figura
    """
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # 1. Serie temporal
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(df.index, df[columna_principal], color='steelblue', linewidth=1)
    ax1.set_title(f'Evolución Temporal de {columna_principal.upper()}', fontweight='bold')
    ax1.set_ylabel('Concentración (µg/m³)')
    ax1.grid(True, alpha=0.3)
    
    # 2. Distribución ICA
    if 'ica_category' in df.columns:
        ax2 = fig.add_subplot(gs[1, 0])
        counts = df['ica_category'].value_counts()
        colors_ica = [ICA_COLORS.get(cat, '#CCCCCC') for cat in counts.index]
        ax2.bar(range(len(counts)), counts.values, color=colors_ica, alpha=0.8)
        ax2.set_xticks(range(len(counts)))
        ax2.set_xticklabels(counts.index, rotation=45, ha='right', fontsize=8)
        ax2.set_title('Distribución Categorías ICA', fontweight='bold')
        ax2.set_ylabel('Frecuencia')
    
    # 3. Histograma
    ax3 = fig.add_subplot(gs[1, 1])
    ax3.hist(df[columna_principal].dropna(), bins=40, color='coral', 
            edgecolor='black', alpha=0.7)
    ax3.set_title(f'Distribución de {columna_principal.upper()}', fontweight='bold')
    ax3.set_xlabel('Concentración (µg/m³)')
    ax3.set_ylabel('Frecuencia')
    
    # 4. Promedios mensuales
    if 'month' in df.columns:
        ax4 = fig.add_subplot(gs[2, 0])
        monthly = df.groupby('month')[columna_principal].mean()
        ax4.bar(monthly.index, monthly.values, color='mediumseagreen', alpha=0.8)
        ax4.set_title('Promedio Mensual', fontweight='bold')
        ax4.set_xlabel('Mes')
        ax4.set_ylabel(f'{columna_principal.upper()} (µg/m³)')
        ax4.set_xticks(range(1, 13))
    
    # 5. Top días contaminados
    ax5 = fig.add_subplot(gs[2, 1])
    daily = df.groupby(df.index.date)[columna_principal].mean().sort_values(ascending=False).head(7)
    ax5.barh(range(len(daily)), daily.values, color='crimson', alpha=0.8)
    ax5.set_yticks(range(len(daily)))
    ax5.set_yticklabels([str(d) for d in daily.index], fontsize=8)
    ax5.set_title('Top 7 Días Más Contaminados', fontweight='bold')
    ax5.set_xlabel(f'{columna_principal.upper()} (µg/m³)')
    ax5.invert_yaxis()
    
    plt.suptitle('Dashboard de Calidad del Aire', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

In [ ]:
create_dashboard_summary(df_clean, columna_principal='pm2_5')